In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
BASE_PATH="/mnt/Archivos/dataset-xray"

In [4]:
from pathlib import Path
from covidframe.tools.load import load_database_from_hdf

In [5]:
base_dir = Path(BASE_PATH)
DEFAULT_DATABASE_NAME_TRAIN = "database_balanced_train.h5"
DEFAULT_DATABASE_NAME_TEST = "database_balanced_test.h5"

In [6]:
df_train = load_database_from_hdf(filename= base_dir / DEFAULT_DATABASE_NAME_TRAIN)
df_test = load_database_from_hdf(filename= base_dir / DEFAULT_DATABASE_NAME_TEST)

In [7]:
category_mappings = {
    "normal":0,
    "pneumonia":1,
    "covid-19":2
}

In [9]:
import numpy as np

In [9]:
X = np.stack(df_train["resized"])

In [10]:
y = df_train["category"].map(category_mappings)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=100)

In [13]:
X_train = X_train.reshape((-1, 299, 299, 1))
X_val = X_val.reshape((-1, 299, 299, 1))

In [8]:
import tensorflow as tf

In [9]:
import tensorflow.compat.v1 as tf1

In [49]:
model_dir=Path("/home/luighi/COVID-Net/models/COVIDNet-CXR4-C")
meta_name="model.meta"
ckpt_name="model-3090"

In [ ]:
#tf1.Session().close()
#doesn't work eager mode
def covidnet_wrapper():
  # Any graph-building code is allowed here.
  tf1.train.import_meta_graph(model_dir / meta_name)

x = import_graph.get_tensor_by_name('input_2:0')
y = import_graph.get_tensor_by_name('softmax/Softmax:0')

# Creates a tf.function with all the imported elements in the function graph.
wrapped_import = tf1.wrap_function(covidnet_wrapper, [])
import_graph = wrapped_import.graph

# Restore the variable values.
tf1.train.Saver(wrapped_import.variables).restore(
    sess=None, save_path= str(model_dir / ckpt_name))

covid_fn = wrapped_import.prune(feeds=x, fetches=y)

In [11]:
X_test = np.stack(df_test["resized"])

In [29]:
df_test["resized_3d"] = df_test["resized"].apply(lambda x: np.tile(x[...,None],3))

In [13]:
from covidframe.tools.image import resize_image

In [30]:
df_test["resized_covidnet"] = df_test["resized_3d"].apply(lambda x: resize_image(x,(480,480)))

In [31]:
X_test_cov = np.stack(df_test["resized_covidnet"])

In [32]:
X_test_cov.shape

(2433, 480, 480, 3)

In [39]:
## For covid net requires medusa backbone image input
df_test["resized_covidnet_medusa"] = df_test["resized"].apply(lambda x: resize_image(x,(256,256)))
df_test["resized_covidnet_medusa"] = df_test["resized_covidnet_medusa"].apply(lambda x: x.astype("float64"))
df_test["resized_covidnet_medusa"] = df_test["resized_covidnet_medusa"].apply(lambda x: x- x.mean())
df_test["resized_covidnet_medusa"] = df_test["resized_covidnet_medusa"].apply(lambda x: x / x.std())
df_test["resized_covidnet_medusa"] = df_test["resized_covidnet_medusa"].apply(lambda x: np.expand_dims(x,-1))

In [40]:
X_test_cov_medusa = np.stack(df_test["resized_covidnet_medusa"])

In [42]:
X_test_cov_medusa[:5].shape

(5, 256, 256, 1)

In [27]:
X_test_cov[:5].shape

(5, 256, 256, 1)

In [50]:
with tf.Graph().as_default() as g:
    meta = tf1.train.import_meta_graph(model_dir / meta_name)
    x = g.get_tensor_by_name('input_1:0')
    y = g.get_tensor_by_name('norm_dense_1/Softmax:0')
    with tf1.Session() as sess:
        meta.restore(sess, str(model_dir / ckpt_name))
        print(sess.run(y, feed_dict={x: X_test_cov[:5]}))

INFO:tensorflow:Restoring parameters from /home/luighi/COVID-Net/models/COVIDNet-CXR4-C/model-3090
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


## Training

In [10]:
df_train["resized_3d"] = df_train["resized"].apply(lambda x: np.tile(x[...,None],3))

In [11]:
del df_test

In [14]:
df_train["resized_covidnet"] = df_train["resized_3d"].apply(lambda x: resize_image(x,(480,480)))

In [ ]:
#https://cs230.stanford.edu/blog/createtrainmodel/

In [ ]:
X_train = np.stack(df_train["resized_covidnet"])

In [ ]:
y_train = df_train["category"].map(category_mappings)

In [56]:
learning_rate=0.1

In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [61]:
with tf.Graph().as_default() as g:
    meta = tf1.train.import_meta_graph(model_dir / meta_name)
    x = g.get_tensor_by_name('input_1:0')
    y = g.get_tensor_by_name('norm_dense_1/Softmax:0')
    image_tensor = g.get_tensor_by_name('input_1:0')
    labels_tensor = g.get_tensor_by_name('norm_dense_1_target:0')
    sample_weights = g.get_tensor_by_name('norm_dense_1_sample_weights:0')
    pred_tensor = g.get_tensor_by_name('norm_dense_1/MatMul:0')
    training_tensor = g.get_tensor_by_name('keras_learning_phase:0')
    
    #Optimizer
    loss_op = tf.reduce_mean(tf1.nn.softmax_cross_entropy_with_logits_v2(
                                logits=pred_tensor, labels=labels_tensor)*sample_weights)
    optimizer = tf1.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
    with tf1.Session() as sess:
        #meta.restore(sess, str(model_dir / ckpt_name))
        sess.run(tf1.global_variables_initializer())
        #print(sess.run(y, feed_dict={x: X_test_cov[:5]}))